In [ ]:
!pip install torch_geometric

Example GNN


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class GraphConvolutionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolutionLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x, adj_matrix):
        aggregated = torch.bmm(adj_matrix, x)

        output = self.linear(aggregated)

        output = F.relu(output)

        return output

class SimpleGraphNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGraphNeuralNetwork, self).__init__()
        self.conv1 = GraphConvolutionLayer(input_dim, hidden_dim)
        self.conv2 = GraphConvolutionLayer(hidden_dim, output_dim)

    def forward(self, x, adj_matrix):
        x = self.conv1(x, adj_matrix)
        x = self.conv2(x, adj_matrix)
        return x

batch_size = 1
num_nodes = 5
input_dim = 10
hidden_dim = 20
output_dim = 30


x = torch.randn(batch_size, num_nodes, input_dim)
print("input tensor : ",x)
adj_matrix = torch.ones(batch_size, num_nodes, num_nodes)

model = SimpleGraphNeuralNetwork(input_dim, hidden_dim, output_dim)

output = model(x, adj_matrix)

print("Output shape:", output.shape)
print("Output tensor : ",output)


input tensor :  tensor([[[-0.9524, -0.0758,  0.8532,  1.7434,  1.7795,  0.2334, -0.5834,
           0.0633, -1.1169, -0.4206],
         [-0.6356,  1.2112, -0.9032,  0.2530,  1.6643, -1.0396,  0.1481,
          -1.3958, -0.1869, -0.2056],
         [-1.7346, -0.5675,  0.8660,  0.0933, -1.5062,  0.4891, -0.2778,
           1.1248, -0.0449, -0.7484],
         [-0.9029, -0.0899, -0.9800, -1.1397,  0.1863, -0.6732,  1.1017,
          -1.0559, -0.6244,  1.2752],
         [ 2.3783, -0.4395,  1.0045,  2.4372,  0.0658,  1.0720, -0.4393,
          -1.0880,  0.3540,  0.6429]]])
Output shape: torch.Size([1, 5, 30])
Output tensor :  tensor([[[0.0000, 2.0330, 4.0297, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0508, 1.5602, 0.0000, 1.6256, 0.0000, 5.9978, 3.5842, 0.1032,
          0.8570, 1.1670, 0.2864, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 3.2850, 0.0000, 0.0070, 0.0000],
         [0.0000, 2.0330, 4.0297, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0508, 

**Implementation**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class MolKGNN(nn.Module):
    def __init__(self, num_node_features, num_kernels, hidden_dim):
        super(MolKGNN, self).__init__()
        self.conv1 = MolConv(num_node_features, hidden_dim, num_kernels)
        self.conv2 = MolConv(hidden_dim * num_kernels, hidden_dim, num_kernels)
        self.lin1 = nn.Linear(hidden_dim * num_kernels, hidden_dim)
        self.lin2 = nn.Linear(hidden_dim, 1)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = torch.sum(x, dim=1)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        x = torch.sigmoid(x)
        return x

class MolConv(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels):
        super(MolConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        self.kernels = nn.Parameter(torch.Tensor(num_kernels, in_channels + 1))
        init.xavier_uniform_(self.kernels)

    def forward(self, x, edge_index):

        assert x.size(1) == self.in_channels, "Input feature dimensions must match in_channels."
        assert edge_index.size(1) == 2, "Edge index must be a 2D tensor with shape (2, num_edges)."

        row, col = edge_index
        x_row = x[row].repeat_interleave(self.num_kernels, dim=0)
        x_col = x[col].repeat_interleave(self.num_kernels, dim=0)
        central_sim = torch.cosine_similarity(x_row, self.kernels[:, :-1], dim=-1)
        neighbor_sim = torch.cosine_similarity(x_col, self.kernels[:, -1:], dim=-1)
        sim = central_sim + neighbor_sim

        return torch.cat([sim[:, k:k+1] * x for k in range(self.num_kernels)], dim=-1)

In [ ]:
import torch
from torch_geometric.data import Data


x = torch.tensor([[6, 0, 0], [1, 0, 0], [8, 0, 1], [6, 0, 0], [1, 0, 0]], dtype=torch.float)
edge_index = torch.tensor([[0, 0, 2, 3, 3], [2, 3, 3, 4, 4]], dtype=torch.long)
y = torch.tensor([1], dtype=torch.float)


data = Data(x=x, edge_index=edge_index, y=y)


print(data)


Data(x=[5, 3], edge_index=[2, 5], y=[1])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch.utils.data import random_split


model = MolKGNN(num_node_features=3, num_kernels=16, hidden_dim=64)


optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

dataset = [data]
train_dataset, test_dataset = random_split(dataset, [len(dataset) - 1, 1])


num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in DataLoader(train_dataset, batch_size=1):
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss}')

model.eval()
with torch.no_grad():
    for test_data in test_dataset:
        out = model(test_data.x, test_data.edge_index)
        prediction = torch.round(out).item()
        print("Predicted label:", prediction)


In [ ]:
import torch
from torch_geometric.data import Data
import torch.optim as optim
from torch_geometric.utils import from_scipy_sparse_matrix


num_nodes = 5
num_node_features = 10
num_kernels = 3
hidden_dim = 20


x = torch.randn(num_nodes, num_node_features)


adj_matrix = torch.ones(num_nodes, num_nodes)


adj_matrix = adj_matrix.to_sparse()


edge_index = from_scipy_sparse_matrix(adj_matrix)


print("Edge index shape:", edge_index.shape)

data = Data(x=x, edge_index=edge_index, edge_attr=None, y=None)


model = MolKGNN(num_node_features, num_kernels, hidden_dim)


optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()


num_epochs = 100
for epoch in range(num_epochs):

    output = model(x, edge_index)


    target = torch.randint(0, 2, (1,))


    loss = criterion(output.squeeze(), target.float())


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print("Training completed!")
